# before you go, take a moment to use the best practices for this repository

This code was run with `python 3.9.4`. The packages are listed at the `requirements.txt` file.

I strongly recommend using `pyenv` to maintain multiple versions of python installed locally. 

## install the virtualenv package on your python via pip

at the terminal just run:

`python -m pip install --user virtualenv`

## create your virtual env. 

Once you have your virtual env created, the packages you once install will not affect your local python packages. Which will avoid conflits. A good practice is to have a virtualenv per project.

The community convention is to use the name `venv` for the environment you are about to create. 

to create your virtualven just type the following command at your terminal:

`virtualenv venv` 

and you should see a `/env/` directory inside your project folder or with the given name you choose at the step before.


## step 1: go to your local repository and type on your terminal:

Warning: Now you should active your `virtualenv`. Keep in mind you need to activate everytime you begin to work with the project. Otherwise, the packages you once install on your virtualenv won't be available to your local python.

#### Linux/Macos

Active your virtualenv using the command source, if you are using linux-like bash. If you are using Windows the command can be diffrent. 

`source venv/bin/activate`

If your terminal looks like the above, you are good.

`(venv) dc@blckjzz pegabot-twitter-api % `

#### For Windows Users

`\env\Scripts\activate.bat`


## step 2: once your virtualenv

Now you should have your virtualenv setup. It's time to install the required packages for running this project.

On terminal at the project folder type:

`pip install -r requirements.txt `


## step 3: create your .env file locally

on terminal, inside the project folder just copy and rename the `.env-example` to `.env`

`$ cp .env.example .env`

### Update your virtualenv packages
#### everytime you need to update your requirements.txt file, you should generate an updated version of your `virtualenv` to a requirements.txt file

Go to your terminal and type

`pip freeze > requirements.txt`

After that, you should commit only your `requirements.txt` file instead your virtualenv `/venv` directory (which is ignored by default on this project on .gitignore file)

## Step 4: Using the virtualenv with Jupyter notebooks

For this you should need a package `ipykernel`, which will allow you to link your local venv with your Jupyter notebook. You should do this in order to use the venv and its packages on your notebook. 

At your terminal type:

`source venv/bin/activate` if not activated

`pip install ipykernel` if you have not installed previously. I added this package to the `requirements.txt` file so you should be fine.

`python -m ipykernel install --user --name=${PWD##*/}`

You should see a message like the above:

`Installed kernelspec pegabot-twitter-api in /Users/dc/Library/Jupyter/kernels/pegabot-twitter-api`

Once you hit the command above, you should restart your Jupyter server to load the new Kernel. You may stop the server or just restart the kernel.

Now you have a new kernel for your jupyter notebooks and should indicate which kernel to use in your project.

To do this go to your project, open any file `.ipynb`. At the upper menu bar find `Kernel > Change Kernel > ` and select your new kernel. If you used the same command as above, the kernel should be named as the same as the project, which is `pegabot-twitter-api` in my case.


## Documentação do Tweepy 
`https://docs.tweepy.org/en/stable/`



In [108]:
# load .env variables from local .env file
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [109]:
# if you wish to reload .env variables
%reload_ext dotenv
%env twitter_api_key

'pFlQlVe3RxDDloIFv3Wq3lv2I'

In [89]:
# project imports listed bellow #

####################################
import os, csv, requests, tweepy
from easydict import EasyDict as edict
import asyncio
####################################

In [54]:
# Loading variables

twitter = {}
twitter['TWITTER_API_KEY'] = %env twitter_api_key 
twitter['TWITTER_API_SECRET'] = %env twitter_api_secret
twitter['TWITTER_API_TOKEN'] = %env twitter_access_token
twitter['TWITTER_API_TOKEN_SECRET'] = %env twitter_access_token_secret

In [13]:
# rewrite the json received from the API to a more readable format
def pegabotTransformResponseJson(pa):
    # recovers all data from twitter data key
    lista = {
        # accessing values from profiles
        'username': pa.profiles[0].username,
        'user_id': pa.twitter_data.user_id,
        'user_name': pa.twitter_data.user_name,
        'created_at': pa.twitter_data.created_at,
        'following': pa.twitter_data.following,
        'followers': pa.twitter_data.followers,
        'number_tweets': pa.twitter_data.number_tweets,
        'mentions': pa.twitter_data.mentions,
        # recoverying values for bot probability
        'result_bot_probability': pa.profiles[0].bot_probability.all,
        'result_bot_probability_info': pa.profiles[0].bot_probability.info,
        # recoverying values for sentiment
        'result_sentiment': pa.profiles[0].language_dependent.sentiment.value,
        'result_network': pa.profiles[0].language_independent.network,
        'result_friends': pa.profiles[0].language_independent.friend,
        'result_temporal': pa.profiles[0].language_independent.temporal,
        'result_user': pa.profiles[0].language_independent.user,
        }
    return edict(lista)

In [104]:
def pegabotAPIRequest(profile, env="local"):
    if env == "prod" or env == "production":
        pegabot_api = "https://backend.pegabot.com.br/botometer"
    else:
        pegabot_api = "http://localhost:9000/botometer"
    try:
        params = {'profile': profile, 'search_for':'profile'}
        response = requests.get(pegabot_api, params)
        # there's a need to treat erros from the Pegabot API.
        # the API should return the profile and a json with an error
        # depending on the error maybe retry the request or just move on to the next line
        if response.status_code == 200:
            jsonResponse = edict(response.json())
            return pegabotTransformResponseJson(jsonResponse)
        else:
            # probably some error
            # usually the user' profile is no longer available
            return edict(response.json())
#        response.raise_for_status()
        # pa.content() # Return the raw bytes of the data payload
        # pa.text() # Return a string representation of the data payload
        # pa.json() # This method is convenient when the API returns JSON
    # Code here will only run if the request is successful
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err) 

In [99]:
# loading profiles from csv file
perfiscsv = []
with open('/Users/dc/Downloads/cristian.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader, None)  # skip the headers
    for row in csv_reader:
        perfiscsv.append(row)

In [114]:
res = pegabotAPIRequest('blckjzz', env="production")

#print(res)

res.result_bot_probability

0.2891630458091107

In [117]:
analise = {}
count = 0
for perfil in perfiscsv:
    #asyncio.create_task()
    print(perfil)
    #analise = pegabotAPIRequest(perfil)
    print(analise)
    count +=1
    if count >= 1:
        break

['Eustacia251']
{}
['VieiraClever1']
{}
['carolin69045611']
{}
['shenybert']
{}
['VavaDone']
{}
['AlziraRosa10']
{}
['de_novaki']
{}
['jorgebrito798']
{}
['Jjjrr70']
{}
['AlexiGa64214227']
{}
['GaboElSoldado']
{}
['CristianoPeix10']
{}
['Carmena79456352']
{}
['william69907498']
{}
['1azevedo_']
{}
['821Claudia']
{}
['Blanca32780063']
{}
['MarclioDiasVie1']
{}
['Teresap1502m']
{}
['saira24338388']
{}
['anzoateguirojo']
{}
['negro20212021']
{}
['eunicecatuchal']
{}
['Vero716']
{}
['Ezequie01455856']
{}
['HctorRu36854512']
{}
['Joseclemente124']
{}
['VelizMariannys']
{}
['RubnDaroQuione2']
{}
['Marisol91870699']
{}
['judsonline']
{}
['cansinesilvage1']
{}
['Keyler93760290']
{}
['Gonza76507998']
{}
['TiaWhatsapp']
{}
['colonbiyenis']
{}
['lesbia_ferrer']
{}
['38Gabrielgb']
{}
['JuniorP40876328']
{}
['iLazulli']
{}
['Lisbeth31982058']
{}
['ginamarudelgad1']
{}
['FernandoLucca13']
{}
['soyjunior01']
{}
['OBruxoeFM_V']
{}
['DigitalG00']
{}
['RSisartex']
{}
['SamuelH24563067']
{}
['enma_pena']

['alexmonbrau']
{}
['Renatosf13']
{}
['AmlivThomas']
{}
['jaykhyl']
{}
['AmbienteSOS']
{}
['athayde_ffc']
{}
['RosaMota79']
{}
['vivianleall']
{}
['coronelsandromg']
{}
['Csargonzalezpe1']
{}
['LenoirFerreira']
{}
['_Nilbis']
{}
['Pensareuminsta1']
{}
['Cristianr290816']
{}
['samsa_anna']
{}
['MJocilei']
{}
['juferraz77']
{}
['PicaPau_UA']
{}
['asitmitra8']
{}
['mmarciano101']
{}
['AJNATERAB']
{}
['marcianevessil']
{}
['Giuseppina666fe']
{}
['MagronesBR']
{}
['pellegrini_alme']
{}
['valeriocunha1']
{}
['fernand56151061']
{}
['vinicius230877']
{}
['MiltonS10226713']
{}
['gonzalesnana']
{}
['DermilsonBarros']
{}
['maffontes']
{}
['LincolndeNEGRE1']
{}
['marjim_r']
{}
['Claudio_Solis']
{}
['marilinha002']
{}
['UlianaMarlene']
{}
['goes_lucimar']
{}
['silvano_leal']
{}
['medeiros_drea']
{}
['robson4P']
{}
['Vict0rCRF']
{}
['Claudio_MFilho']
{}
['rwtrader1']
{}
['chagasfc1']
{}
['Sant0Gabriel7']
{}
['SenadorRogerio']
{}
['FelixPolonio']
{}
['uirajan']
{}
['julianopsol']
{}
['ldpichurilo']
{

{}
['Maritza_81']
{}
['mahcoconut']
{}
['TvDNewsBrasil']
{}
['Siberio_1']
{}
['Ricardo85802392']
{}
['JavierJoseBerm4']
{}
['juntosomoforte']
{}
['Rc81Vzla']
{}
['AvilaLigmary']
{}
['MaryJos94960820']
{}
['roquecontreras9']
{}
['AlirioPerozi']
{}
['Mariane52485539']
{}
['Lin16859363']
{}
['WilliamsVeliz11']
{}
['RafaelIzturiz']
{}
['Carlosc42125355']
{}
['Guilher70958189']
{}
['Reginap49492018']
{}
['onepe14']
{}
['Maga19511']
{}
['AntonioPrezAl10']
{}
['fernan0706']
{}
['2CJdmendzr71']
{}
['xioriera56']
{}
['Morelba66746188']
{}
['Angel58552841']
{}
['Miriam_lugo23']
{}
['Daniell86390710']
{}
['lucia91473857']
{}
['guitomiasi']
{}
['ALSM987']
{}
['WilkarLo']
{}
['RLamigueiro']
{}
['LUCILA81431780']
{}
['Luisflo24304822']
{}
['JosAlva80012723']
{}
['jhanaleite']
{}
['Kaleb159z']
{}
['Yosmarycandanga']
{}
['pastranosoto']
{}
['Disney69452936']
{}
['Jhonceballo6']
{}
['Gloria32548871']
{}
['esther04965523']
{}
['OscarVa07704273']
{}
['Yerr55733832']
{}
['YulitzaGuerrera']
{}
['yoaluisio'

['eduardozim']
{}
['Miguel_Matoma']
{}
['gabrielmacaco27']
{}
['Estefrivx']
{}
['TiagoBaldezz']
{}
['JorgeHe94070137']
{}
['laurianoslvra']
{}
['novaiscicero1']
{}
['montoeira']
{}
['juristadofuturo']
{}
['8_zaidaE']
{}
['Yajaira698']
{}
['filgueiras85']
{}
['cotty0306']
{}
['jones_egydio']
{}
['michelsouza25']
{}
['FaimaLima']
{}
['DanielSurtado']
{}
['marcelodestaque']
{}
['nogeira_s']
{}
['leomar1972']
{}
['yogensy18']
{}
['electrico43']
{}
['GaaboPR']
{}
['CamposMilagros']
{}
['pavao_helio']
{}
['RosanaCamargod1']
{}
['zizonavarro']
{}
['Buensancho_']
{}
['mahebhagat']
{}
['PinzonRosmary']
{}
['joseluisdj']
{}
['FedoraAbdala3']
{}
['ffredel']
{}
['gomezportuguesa']
{}
['Fernand98756869']
{}
['deysangn']
{}
['NiviaCabana']
{}
['LinaresHomero']
{}
['RaulCarrascal8']
{}
['SoloopinandoMl']
{}
['LTAGLI']
{}
['Miranda110282']
{}
['Jonnath91938228']
{}
['Edson_Oliveira']
{}
['bhupenverma71']
{}
['Nobelys2']
{}
['Karimmeza']
{}
['porras_f']
{}
['carlos_stat']
{}
['vtarantelli']
{}
['chilit

{}
['noliusiline']
{}
['BruBauna']
{}
['4AlphabetBot']
{}
['otaviano367_']
{}
['Val_King_Robot']
{}
['Pedro_AlvesNeto']
{}
['andrezamatais']
{}
['JiParanaOficial']
{}
['LucioVasconcel5']
{}
['PauloRoscoe']
{}
['guiskd13']
{}
['c_vicente1965']
{}
['edssonrodrigues']
{}
['abelgsouza65']
{}
['lucivanibarreto']
{}
['Srgio45974011']
{}
['FabioRibeiroAl3']
{}
['djagues']
{}
['1_tau_de_edu']
{}
['Rebolsominia']
{}
['wanderleybh']
{}
['BrunoCa11583762']
{}
['filipe_augustus']
{}
['jean_jeyson']
{}
['AAndersonbm']
{}
['Carlos36cCarlos']
{}
['MariWaltrick51']
{}
['Luis_R_Oficial']
{}
['reginatto38']
{}
['albcorrales']
{}
['IzhGIZwb9hqB9dZ']
{}
['andrecmoura']
{}
['lamego_glorinha']
{}
['larcmar73']
{}
['Robson871802851']
{}
['FernandoLaste']
{}
['SicTadeu']
{}
['William62494033']
{}
['valmirtav']
{}
['A2J97981339']
{}
['magnonialexandr']
{}
['clarediportela']
{}
['JulioK70']
{}
['Luis_Gome1']
{}
['pastorangolar']
{}
['carlosmaltz']
{}
['Marcelo49268583']
{}
['Fernandapere']
{}
['BeaRousseff']
{}

...

{}
['LulaLucena']
{}
['Regina92915089']
{}
['kentarogon']
{}
['jonatasantos90']
{}
['Flor_PatriotaBR']
{}
['RobsonRufino14']
{}
['Puebla3_0']
{}
['EudesRo30563914']
{}
['JonhFdt']
{}
['CorteFossard']
{}
['MauWM']
{}
['Vthsa_']
{}
['CristianeCaire8']
{}
['cowboy4u22']
{}
['joaosilvestref']
{}
['pameryco']
{}
['Marcos_BarbosaL']
{}
['SolAthayde']
{}
['izacrispi111']
{}
['MARCELOZANI14']
{}
['mariado69326861']
{}
['Redragon974']
{}
['leda_coimbra']
{}
['jluiscava']
{}
['rurusoave']
{}
['ANTONIOLLI2022']
{}
['wawembiird']
{}
['TomPettinger1']
{}
['RomanoDriver13']
{}
['vania37']
{}
['suelisl']
{}
['2611Deepanshu']
{}
['waspaiva']
{}
['paulosensato1']
{}
['RochaValnei']
{}
['venator_3']
{}
['veigaDi']
{}
['magneto1976']
{}
['entaotabomentao']
{}
['GlaucioCampos']
{}
['EdvalVozao']
{}
['_dedepds_']
{}
['PauloCe04748414']
{}
['RicardoAdriao1']
{}
['14rogerio14']
{}
['FabioBRA_SEP']
{}
['monicasouza321']
{}
['agnaldo_crist']
{}
['esamico']
{}
['elias_etelvino']
{}
['irfhabib']
{}
['coelhofc81'

{}
['helena_eyer']
{}
['Luciano11310764']
{}
['FMagalhae']
{}
['PriRodriguesFla']
{}
['LAQMHR']
{}
['andreluisstahl']
{}
['MR3ann']
{}
['juhmclean']
{}
['_Rafael_AA_']
{}
['juliana_nasx']
{}
['brasileiro1985']
{}
['PTfobia']
{}
['1lusquinha_']
{}
['das_soares']
{}
['raphozzz_']
{}
['welcomealec']
{}
['AvilaTome']
{}
['brasileus']
{}
['afternats']
{}
['Kistherra']
{}
['SilvaMarli0']
{}
['IRLRaiden1306']
{}
['edneicandido']
{}
['DeboraSantinni']
{}
['iamJorgeLucas']
{}
['Dinilton6']
{}
['usharani_33']
{}
['ThiagoM80448067']
{}
['SrgioOl88544949']
{}
['NicolleMood']
{}
['worldwidecenso1']
{}
['flaflww']
{}
['Laila26165439']
{}
['arthurteixeiraa']
{}
['gmvliff']
{}
['Eugenioramoscx']
{}
['PauloGu96822531']
{}
['SIDENIL']
{}
['JMarianni']
{}
['ivinnyr']
{}
['anamaria_13_']
{}
['andpatriota']
{}
['risomarbcarneir']
{}
['_kim_honey_']
{}
['DerHauptmann81']
{}
['gutomaaciel']
{}
['gabie_fr']
{}
['AACToni']
{}
['SetarkFeyGothic']
{}
['Mjdss86Offred86']
{}
['AlanPereira603']
{}
['LUHCORSSI']
{}


['bebert06480']
{}
['DionorF']
{}
['dissePauIo']
{}
['FlavioBocaGyn']
{}
['ZagoSandro']
{}
['arrobajeansilva']
{}
['anevaldo81']
{}
['Marciojjm']
{}
['SoBarreto']
{}
['tiago_jsp']
{}
['SerafimmeloF']
{}
['CarmemMontes']
{}
['Francis10234290']
{}
['daniceruttiso']
{}
['Rqguel']
{}
['M3dus4_s']
{}
['vitordaleiroo']
{}
['CarlaSa19890720']
{}
['JuraJura75']
{}
['GLADYSARRUDA']
{}
['ArmelindoPasso2']
{}
['EvgCris']
{}
['YwersonLM']
{}
['JulioC_caldas']
{}
['MarcelFrana3']
{}
['JuliusSanctus']
{}
['v_mangabeira']
{}
['Marcos13Silva1']
{}
['misslabafero']
{}
['CarmemAraujodo1']
{}
['MOREIRARJFLA1']
{}
['camilazarur']
{}
['palmeiranderson']
{}
['brunopivan']
{}
['trash_era']
{}
['CaioTeruel']
{}
['Adrianojapao3']
{}
['oalexandredias']
{}
['ekocidadao_18']
{}
['acarlosbiten']
{}
['Edi519']
{}
['DeiseQuintilia1']
{}
['Xavier1William']
{}
['paulocesarsil15']
{}
['FXGT_official']
{}
['rubensborcem']
{}
['umbombeck']
{}
['RLuttzer']
{}
['_FLima_']
{}
['elaeconvicta']
{}
['tarcisojosedeli']
{}
['Ags

{}
['eduburti']
{}
['FarahCubas']
{}
['guydbennett']
{}
['coverteconomics']
{}
['eldermda']
{}
['bruthabitcoin']
{}
['Alcindodutra']
{}
['MonicaPicc2021']
{}
['ChadTripathi']
{}
['lokodw']
{}
['Erabhishekarya1']
{}
['RogerFe10528483']
{}
['LeverageLenny']
{}
['KallasJe']
{}
['FORALLCRYPT0']
{}
['GFGavioli']
{}
['tsuki45451919']
{}
['clemiltoncunhas']
{}
['berkeleyguy98']
{}
['naclout']
{}
['Gordondeosasco']
{}
['itz_krish007']
{}
['Tk_GomesDark']
{}
['MorgaVitel']
{}
['PedroGentileza']
{}
['Mana_Luft']
{}
['legomexs']
{}
['robecker71']
{}
['2bPatto']
{}
['emersonnatal']
{}
['johnybrasa']
{}
['japadonorte']
{}
['Rapunzelsdairy']
{}
['chicopedroca']
{}
['maiavilhos4']
{}
['Ricardo19376147']
{}
['lenonazariob']
{}
['IcornMr']
{}
['MonstreGaletes']
{}
['benji_man55']
{}
['yasnaovoumais']
{}
['dodaferrari']
{}
['Barreto381']
{}
['Danbiohackinman']
{}
['REBEL45d']
{}
['0x1428']
{}
['MIGUELVGN881']
{}
['oitaurina']
{}
['tysonsilv4']
{}
['RYO_RGA_0303']
{}
['BipBurke']
{}
['LeonDireita']
{}
['

['FrancaPerpetua']
{}
['Nivaldomotacrb']
{}
['maridefreitasf']
{}
['JMBB38']
{}
['perrechato']
{}
['Capixabawr1']
{}
['tamil_marx']
{}
['trendsinAI']
{}
['Ibragim35646035']
{}
['novusisaacus']
{}
['redecompolicial']
{}
['SudhaBharti2020']
{}
['Estefanocruzei1']
{}
['MariaKmm2324']
{}
['Luiza83525048']
{}
['FelipeCaramori']
{}
['hikarupisu_756']
{}
['RobsonPassarel1']
{}
['julessteps']
{}
['DeguthiReserva']
{}
['AntifaRJ1']
{}
['marcosfreitas_']
{}
['renatod7922']
{}
['jimmyso01649130']
{}
['Bia06412499']
{}
['MecksWorld']
{}
['CarlosE76474912']
{}
['xandiscapini']
{}
['justin00800']
{}
['marcosgl28']
{}
['llwspslqt']
{}
['Julianapedemou1']
{}
['dan_panteera']
{}
['delvecchi']
{}
['pr4refletir']
{}
['damiao_dimas']
{}
['ravindranathm13']
{}
['marisaleite']
{}


In [17]:
# TODO: transform Pegabot Request assync and make multiple request to it.
#profiles = ['blckjzz']
for p in profiles:
    profile = pegabotAPIRequest(p)
    
print(profile)

{'username': 'blckjzz', 'user_id': '1080370887357071362', 'user_name': 'Diego Cerqueira', 'created_at': 'Wed Jan 02 07:51:06 +0000 2019', 'following': 363, 'followers': 294, 'number_tweets': 261, 'mentions': ['RosieDaSerenata', 'cuducos', 'NataliaMazotte', 'teomoura2', 'pegabots', 'CodingRights', 'ninadhora', 'fecampa', 'boomartins', 'caiocvm', 'AvHStiftung', 'WZB_Berlin', 'AnatelGovBR', 'ClaroBrasil', 'eduardopaes', 'minsaude', 'MaluMondelli', 'Jgb_santos', 'ITSriodejaneiro', 'ComuNICbr', 'vivobr', 'prenass', 'pblnns', 'Rioluz', 'tarciziosilva', 'CNNBrasil', 'ruha9', 'SueliCarneiro', 'samadeu', 'passalanorh', 'gomarigomes', 'dantasglends', 'silvanabahia', 'flamedeirosss', 'educamidia', 'tvcultura', 'AfrOyaTech', 'ThayaneGuim', '_Laris_Lari', 'destemida_yara', 'YouTube', 'AnarcoFino', 'blckjzz', 'bajpaes', 'yaso', 'eniolourenco', 'cdr_br', 'instagram', 'redescordiais', 'pqgustavo', 'caiocgomes'], 'result_bot_probability': 0.2891164029162831, 'result_bot_probability_info': '<p>Um dos cr